In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
#IMporting the itents json file for the chatbot
with open('/kaggle/input/intents-for-chabot/intents.json') as json_data:
    intents = json.load(json_data)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [15]:
stemmer = LancasterStemmer()
words = []
classes=[]
documents = []
ignore = ['?']
#NOW loop through each sentence in the intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        #add word to the word list
        words.extend(w)
        #add words to the documents
        documents.append((w, intent['tag']))
        #add tags to out classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
#PErform stemming and lower each word as well as remove the duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

#NOW remove the duplicates
classes = sorted(list(set(classes)))

print(len(documents),'documents')
print(len(classes),'classes',classes)
print(len(words),'unique stemmed words',words)

31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [ ]:
#Now we create the training data
training = []
output = []

#creating an empty array for output
output_empty = [0] * len(classes)

#Creating training set of bag of words for each sentence
for doc in documents:
    #Initialize bag of words
    bag = []
    #list of tokenized words for the pattern
    pattern_words = doc[0]
    #Stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    #create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
#Shuffling features and turning it into np.array
random.shuffle(training)
training = (training)

#creating training lists
train_x = list(training[:,0])
trian_y = list(training[:,1])


In [ ]:
#NOw reseting underlying graph data
tf.reset_default_graph()

#building nueral network
net = tflearn.input_data(shape=[None,len(train_x[0])])
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#starting the training now
model.fit(train_x,train_y,n_epochs=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')


In [ ]:
import pickle
pickle.dump({'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open('training_data','wb'))

#restoring all the data structures
data = pickle.load(open('training_data','rb'))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
#now load the saved data
model.load('.model.tflearn')

def clean_up_sentence(sentence):
    #tokenize the pattern
    setence_words = nltk.word_tokenize(sentence)
    #stemming each word
    sentence_words =[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

#now returning the bag of words array :0 or 1 for each word in the bag that exists
def bow(sentence,words,show_details=False):
    #tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    #generating the bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print('found in bag: %s' % w)
    return(np.array(bag))



ERROR_THRESHOLD = 0.30
def classify(sentence):
    #generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    #filter out the predictions below the threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    #sort by the strength of porbability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    #return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    #if we have a classification then find the matching intent tag
    if results:
        #loop as long as there are matchces to the process
        while results:
            for i in intents['intents']:
                #find a tag matching the first result
                if i['tag']== results[0][0]:
                    #a random response from the intent
                    return print(random.choice(i['response']))
            results.pop(0)
            